In [ ]:
import pandas as pd
import json
import numpy as np
import math

def process_reservoir_data(input_file, output_file):
    """
    Process reservoir data for the dashboard app
    """
    # Load data
    df = pd.read_csv(input_file)
    
    # Replace -999.9 with None and handle NaN values
    df = df.replace(-999.9, None)
    df = df.replace(-99, None)
    
    # Convert NaN to None for proper JSON serialization
    df = df.where(pd.notnull(df), None)
    
    # Clean and validate coordinates
    df = df.dropna(subset=['longitude', 'latitude'])
    df = df[(df['longitude'] >= -180) & (df['longitude'] <= 180)]
    df = df[(df['latitude'] >= -90) & (df['latitude'] <= 90)]
    
    # Add classification categories
    df['is_critical_reservoir'] = df['max_capacity_mcm'].fillna(0) > 100
    df['is_critical_hydropower'] = df['power_mw'].fillna(0) > 30
    
    # Add capacity categories for visualization
    def get_capacity_category(capacity):
        if pd.isna(capacity) or capacity <= 0:
            return 'Unknown'
        elif capacity <= 10:
            return '0-10MCM'
        elif capacity <= 100:
            return '10-100MCM'
        elif capacity <= 1000:
            return '100-1000MCM'
        else:
            return '>1000MCM'
    
    def get_power_category(power):
        if pd.isna(power) or power <= 0:
            return 'Unknown'
        elif power <= 30:
            return '0-30MW'
        elif power <= 100:
            return '30-100MW'
        elif power <= 1000:
            return '100-1000MW'
        else:
            return '>1000MW'
    
    df['capacity_category'] = df['max_capacity_mcm'].apply(get_capacity_category)
    df['power_category'] = df['power_mw'].apply(get_power_category)
    
    # Create summary statistics
    summary = {
        'total_count': len(df),
        'countries': df['country'].value_counts().to_dict(),
        'main_uses': df['main_use'].value_counts().to_dict(),
        'capacity_distribution': df['capacity_category'].value_counts().to_dict(),
        'power_distribution': df['power_category'].value_counts().to_dict(),
        'critical_reservoirs': int(df['is_critical_reservoir'].sum()),
        'critical_hydropower': int(df['is_critical_hydropower'].sum())
    }
    
    # Prepare final dataset
    processed_data = {
        'features': df.to_dict('records'),
        'summary': summary,
        'metadata': {
            'total_records': len(df),
            'last_updated': pd.Timestamp.now().isoformat(),
            'filters': {
                'countries': sorted(df['country'].dropna().unique().tolist()),
                'main_uses': sorted(df['main_use'].dropna().unique().tolist()),
                'commission_years': {
                    'min': int(df['commission_year'].min()) if not df['commission_year'].isna().all() else None,
                    'max': int(df['commission_year'].max()) if not df['commission_year'].isna().all() else None
                }
            }
        }
    }
    
    def clean_nans(obj):
        """
        Recursively convert all NaN, inf, -inf to None for JSON safety
        """
        if isinstance(obj, float):
            if math.isnan(obj) or math.isinf(obj):
                return None
            return obj
        elif isinstance(obj, dict):
            return {k: clean_nans(v) for k, v in obj.items()}
        elif isinstance(obj, list):
            return [clean_nans(v) for v in obj]
        else:
            return obj

    # Clean processed_data recursively
    safe_data = clean_nans(processed_data)

    # Strict JSON output (disallow NaN)
    with open(output_file, 'w') as f:
        json.dump(safe_data, f, indent=2, allow_nan=False)

    # print(f"✅ JSON cleaned and saved safely to {output_file}")
    
    print(f"✅ Processed {len(df)} records")
    print(f"✅ Data saved to {output_file}")
    print(f"📊 Summary: {summary['critical_reservoirs']} critical reservoirs, {summary['critical_hydropower']} critical hydropower stations")
    
    return processed_data

if __name__ == "__main__":
    # Process the data
    result = process_reservoir_data(
        # input_file='14-reservoirs_30MW_and_100MCM.csv',
        input_file='13-allData_addSEAWEA_ID.csv',
        output_file='processed_reservoir_data.json'
    )

✅ Processed 369 records
✅ Data saved to processed_reservoir_data.json
📊 Summary: 188 critical reservoirs, 262 critical hydropower stations


C:\Users\15330\AppData\Local\Temp\ipykernel_23720\4001473938.py:26: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['is_critical_reservoir'] = df['max_capacity_mcm'].fillna(0) > 100
C:\Users\15330\AppData\Local\Temp\ipykernel_23720\4001473938.py:27: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['is_critical_hydropower'] = df['power_mw'].fillna(0) > 30
